In [2]:
from google.colab import drive
drive.mount( '/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/'My Drive'/camouflaged-object-detection

/content/gdrive/My Drive/camouflaged-object-detection


In [4]:
!unzip COD10K-v3.zip
# import shutil
# shutil.unpack_archive("COD10K-v3.zip", "dataset")

Streaming output truncated to the last 5000 lines.
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-24-Caterpillar-1649.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-24-Caterpillar-1651.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-24-Caterpillar-1654.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1656.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1657.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1660.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1661.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1662.jpg  
 extracting: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1664.jpg  
 extracting: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1665.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1666.jpg  
  inflating: COD10K

In [3]:
% cd /content/gdrive/'My Drive'/camouflaged-object-detection/
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd  ..

/content/gdrive/My Drive/camouflaged-object-detection
/content/gdrive/My Drive/camouflaged-object-detection/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-28lmxvd_
Created temporary directory: /tmp/pip-req-tracker-13jel8ec
Created requirements tracker '/tmp/pip-req-tracker-13jel8ec'
Created temporary directory: /tmp/pip-install-bgcqfjsj
Processing /content/gdrive/My Drive/camouflaged-object-detection/apex
  Created temporary directory: /tmp/pip-req-build-q70a7ovs
  Added file:///content/gdrive/My%20Drive/camouflaged-object-detection/apex to build tracker '/tmp/pip-req-tracker-13jel8ec'
    Running setup.py (path:/tmp/pip-req-build-q70a7ovs/setup.py) egg_info for package from file:///content/gdrive/My%20Drive/camouflaged-object-detecti

In [ ]:
# Training model
%cd /content/gdrive/'My Drive'/camouflaged-object-detection
import torch
from src.SINet import SINet_ResNet50
from src.dataloader import get_loader
from src.trainer import trainer, adjust_lr
from apex import amp

class P:
  def __init__(self):
    self.train_img_dir = 'COD10K-v3/Train/Image/'
    self.train_gt_dir = 'COD10K-v3/Train/GT_Object/' # ground truth
    self.epochs = 40
    self.lr = 1e-4 # initial learning rate
    self.batchsize = 36

    self.trainsize = 352 # size of image for training, using smaller resolution for speed
    self.clip_margin = 0.5 # gradient clipping margin
    self.decay_rate = 0.1 # rate of lr decay per decay step
    self.decay_epoch = 30 # decay lr every N epochs

    self.save_epoch = 10 # save model weights after every N epochs
    self.save_model = './Snapshot_SINet/'
    self.gpu = 0

opt = P()
torch.cuda.set_device(opt.gpu)

# 
model = SINet_ResNet50(channel=32).cuda()
print(model)

optimizer = torch.optim.Adam(model.parameters(), opt.lr)
LogitsBCE = torch.nn.BCEWithLogitsLoss()

net, optimizer = amp.initialize(model, optimizer, opt_level='O1')

train_loader = get_loader(opt.train_img_dir, opt.train_gt_dir, batchsize=opt.batchsize,
                          trainsize=opt.trainsize, num_workers=12)
total_step = len(train_loader)

print('-' * 30, "\n[Training Dataset INFO]\nimg_dir: {}\ngt_dir: {}\nLearning Rate: {}\nBatch Size: {}\n"
                "Training Save: {}\ntotal_num: {}\n".format(opt.train_img_dir, opt.train_gt_dir, opt.lr,
                                                          opt.batchsize, opt.save_model, total_step), '-' * 30)

for epoch in range(1, opt.epochs+1):
    adjust_lr(optimizer, epoch, opt.decay_rate, opt.decay_epoch)
    trainer(train_loader=train_loader, model=model,
            optimizer=optimizer, epoch=epoch,
            opt=opt, loss_func=LogitsBCE, total_step=total_step)

In [2]:
## 352x352 and 64 channels
%cd /content/gdrive/'My Drive'/camouflaged-object-detection/src
!python train.py 

/content/gdrive/My Drive/camouflaged-object-detection/SINet
[INFO] initialize weights from resnet50
------------------------------ SINet_ResNet50(
  (resnet): ResNet_2Branch(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256

In [2]:
## 352x352 and 32 channels
%cd /content/gdrive/'My Drive'/camouflaged-object-detection/src
!python train.py 

/content/gdrive/My Drive/camouflaged-object-detection/SINet
[INFO] initialize weights from resnet50
------------------------------ SINet_ResNet50(
  (resnet): ResNet_2Branch(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256

In [3]:
!pip install scipy==1.2.2

     |████████████████████████████████| 24.8MB 137kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.2.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
# Testing 

import torch
import torch.nn.functional as F
import numpy as np
import os
import argparse
from scipy import misc
from src.SINet import SINet_ResNet50
from src.dataloader import test_dataset
from src.trainer import eval_mae, numpy2tensor

class P:
  def __init__():
    self.testsize = 352 # input size
    self.model_path = './Snapshot_SINet/SINet_40.pth'
    self.test_save = './Result_SINet/'

opt = P()


model = SINet_ResNet50().cuda()
model.load_state_dict(torch.load(opt.model_path))
model.eval()

for dataset in ['COD10K-v3']:
    save_path = opt.test_save + dataset + '/'
    os.makedirs(save_path, exist_ok=True)
    # NOTES:
    #  if you plan to inference on your customized dataset without grouth-truth,
    #  you just modify the params (i.e., `image_root=your_test_img_path` and `gt_root=your_test_img_path`)
    #  with the same filepath. We recover the original size according to the shape of grouth-truth, and thus,
    #  the grouth-truth map is unnecessary actually.
    test_loader = test_dataset(image_root='../dataset/{}/Test/Image/'.format(dataset),
                               gt_root='../dataset/{}/Test/GT_Object/'.format(dataset),
                               testsize=opt.testsize)
    img_count = 1
    for iteration in range(test_loader.size):
        # load data
        image, gt, name = test_loader.load_data()
        gt = np.asarray(gt, np.float32)
        gt /= (gt.max() + 1e-8)
        image = image.cuda()
        # inference
        _, cam = model(image)
        # reshape and squeeze
        cam = F.upsample(cam, size=gt.shape, mode='bilinear', align_corners=True)
        cam = cam.sigmoid().data.cpu().numpy().squeeze()
        # normalize
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        misc.imsave(save_path+name, cam)
        # evaluate
        mae = eval_mae(numpy2tensor(cam), numpy2tensor(gt))
        # coarse score
        print('[Eval-Test] Dataset: {}, Image: {} ({}/{}), MAE: {}'.format(dataset, name, img_count,
                                                                           test_loader.size, mae))
        img_count += 1

print("\n[Congratulations! Testing Done]")

In [4]:
!ls

EvaluationTool	MyTest.py   README.md	     Result    Src
Images		MyTrain.py  requirement.txt  Snapshot


In [6]:
!python MyTest.py --model_path ./Snapshot/2020-CVPR-SINet/SINet_20.pth --test_save ./Result/2020-CVPR-SINet/ --testsize 352

[INFO] initialize weights from resnet50
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
[Eval-Test] Dataset: COD10K-v3, Image: COD10K-CAM-1-Aquatic-1-BatFish-2.png (1/4000), MAE: 0.10529448091983795
[Eval-Test] Dataset: COD10K-v3, Image: COD10K-CAM-1-Aquatic-1-BatFish-4.png (2/4000), MAE: 0.1899843066930771
[Eval-Test] Dataset: COD10K-v3, Image: COD10K-CAM-1-Aquatic-1-BatFish-5.png (3/4000), MAE: 0.08756836503744125
[Eval-Test] Dataset: COD10K-v3, Image: COD10K-CAM-1-Aquatic-1-BatFish-6.png (4/4000), MAE: 0.03850087523460388
[Eval-Test] Dataset: COD10K-v3, Image: COD10K-CAM-1-Aquatic-10-LeafySeaDragon-416.png (5/4000), MAE: 0.08694673329591751
[Eval-Test] Dataset: COD10K-v3, Image: COD10K-CAM-1-Aquatic-10-LeafySeaDragon-419.png (6/4000), MAE: 0.15902426838874817
[Eval-Test] Dataset: 

In [9]:
# Taken output of above as a string and extracted the MAE values for analysis
# import numpy as np
# T = s.split("\n")
# len(T)
# D = np.array(list(map(lambda x: eval(x.split()[-1]), T)))
print("MAE on the test set of 4000 unseen images: ", np.mean(D))
# ^ only for 20 epochs. for full run of 40 epochs, it would have been way better.

MAE on the test set of 4000 unseen images:  0.09132068389677443


In [ ]:
# got corrupt due to gradient overflow
# !python MyTest.py --model_path ./Snapshot/2020-CVPR-SINet/SINet_30.pth --test_save ./Result/2020-CVPR-SINet/ --testsize 352